# Sentinel-2 (global) <img align="right" src="../../resources/easi_logo.jpg">

#### Index
- [Overview](#Overview)
- [Setup (dask, imports, query)](#Setup)
- [Product definition (measurements, flags)](#Product-definition)
- [Quality layer (mask)](#Quality-layer)
- [Scaling and nodata](#Scaling-and-nodata)
- [Visualisation](#Visualisation)
- [Optional: Reproject](#Optional:-Reproject)
- [Appendix](#Appendix)

## Overview

Sentinel-2 is an Earth observation mission from the EU Copernicus Programme that systematically acquires optical imagery at high spatial resolution (up to 10 m for some bands). The mission is a constellation of two identical satellites in the same orbit, 180° apart for optimal coverage and data delivery. Together, they cover all Earth's land surfaces, large islands, inland and coastal waters every 3-5 days.

Sentinel-2A was launched on 23 June 2015 and Sentinel-2B followed on 7 March 2017.
Both of the Sentinel-2 satellites carry a wide swath high-resolution multispectral imager with 13 spectral bands.
For more information on the Sentinel-2 platforms and applications see the [European Space Agency website](http://www.esa.int/Applications/Observing_the_Earth/Copernicus/Overview4).

_Adapted from https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/DEA_datasets/Sentinel_2.ipynb_

#### Data source and documentation

EASI provides at least four Sentinel-2 products from three sources of data. These represent Analysis Ready Data (ARD) products corrected to surface reflectance.

We describe in this notebook the products that can be used in a global context. That is, products processed with globaly applicable algorithms.

| Name | Product | Information |
|------|---------|-------------|
| Sentinel-2 COGs | `s2_l2a` | - Direct indexing from Sentinel-2 Cloud-Optimized GeoTIFFs. [More information below](#Sentinel-2-Cloud-Optimized-GeoTIFFs)<br>- Use for global AOIs where Sen2cor surface correction is required |

#### EASI pipeline

| Task | `s2_l2a` |
|------|------------------------|
| Source | https://registry.opendata.aws/sentinel-2-l2a-cogs |
| Download | Index in place |
| Prepare | stac-to-dc |
| TODO | Update "band_0X" aliases<br>Capitalise SLC flag #9 |

## Setup

#### Imports

In [ ]:
# Basic plots
%matplotlib inline
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Common imports and settings
import os, sys
os.environ['USE_PYGEOS'] = '0'
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr

# Datacube
import datacube
from datacube.utils.rio import configure_s3_access
from datacube.utils import masking
from datacube.utils.cog import write_cog
# https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools
from dea_tools.plotting import display_map, rgb
from dea_tools.datahandling import mostcommon_crs

# EASI defaults
easinotebooksrepo = '/home/jovyan/easi-notebooks'
if easinotebooksrepo not in sys.path: sys.path.append(easinotebooksrepo)
from easi_tools import EasiDefaults, xarray_object_size, notebook_utils
from easi_tools.s2l2a_patch import load_s2l2a_with_offset_patch

In [ ]:
# Data tools
import numpy as np
from datetime import datetime

# Datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

# Dask
from dask.distributed import Client, LocalCluster

#### EASI defaults

In [ ]:
easi = EasiDefaults()

family = 'sentinel-2'
product = easi.product(family)
display(Markdown(f'Default {family} product for "{easi.name}": [{product}]({easi.explorer}/products/{product})'))

#### Dask cluster

In [ ]:
# Local cluster
# Default is to run on a compute node with 28 GiB of available memory and 8 cores.
# We'll make that explicit here .. but this should be adjusted based on your workflow

cluster = LocalCluster(n_workers=2, threads_per_worker=4)
cluster.scale(n=2, memory="14GiB")
client = Client(cluster)
display(client)

dashboard_address = notebook_utils.localcluster_dashboard(client=client,server=easi.hub)
display(dashboard_address)

#### ODC database

In [ ]:
dc = datacube.Datacube()

# Access AWS "requester-pays" buckets
# This is necessary for reading data from most third-party AWS S3 buckets such as for Landsat and Sentinel-2
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

## Example query

Change any of the parameters in the `query` object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product:

In [ ]:
print(f'{easi.explorer}/products/{product}')

In [ ]:
# Vietnam
min_longitude, max_longitude = (105.5, 106.4)
min_latitude, max_latitude = (9.2, 10.0)
min_date = '20y20-05-31'
max_date = '2020-06-01'

query = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
    'time': (min_date, max_date),           # Any parsable date strings
}

#### Most common CRS

Sentinel-2 datasets are stored with different coordinate reference systems (CRS), corresponding to multiple UTM zones used for S2 L1B tiling. S2 measurement bands also have different resolutions (10 m, 20 m and 60 m). As such S2 queries need to include the following two query parameters:

* `output_crs` - This sets a consistent CRS that all Sentinel-2 data will be reprojected to, irrespective of the UTM zone the individual image is stored in.
* `resolution` - This sets the resolution that all Sentinel-2 images will be resampled to. 

Use `mostcommon_crs()` to select a CRS. Adapted from https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/Tools/dea_tools/datahandling.py

In [ ]:
# Most common CRS
native_crs = notebook_utils.mostcommon_crs(dc, query)
print(f'Most common native CRS: {native_crs}')

# Target xarray params
# Usually we group input scenes on the same day to a single time layer
# but we need to apply the offset to each input scene
# so we will "group by day" after the merge.
load_params = {
    'measurements': ['blue', 'red', 'nir', 'scl'],
    'output_crs': native_crs,               # EPSG code
    'resolution': (-20, 20),                # Target resolution
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'x': 2048, 'y': 2048},  # Dask chunks
}

#### Apply the correct _Offset_ to the source data

ESA introduced a change to their L1C processing that encodes their L1C and L2A products with an _Offset_ value such that
`real_value = encoded_value * scale_factor + offset`

Cloud data providers of ESA's S2A data additionally may have pre-applied the offset or not to the data that we index and load.

This introduces an inconsistency in the S2A series that we need to account for.

The scale and offset values, whether applied or not, are consistent.

1. For each of the _datasets_ corresponding to a query, filter into lists of *offset-applied* and *offset-not-applied*
1. Load each list into separate xarray Datasets
1. Apply the _offset_ to the respective xarray Dataset
1. Merge the two xarray Datasets
1. Continue to apply masking and analytics as usual.

These steps are undertaken in `load_s2l2a_with_offset_patch`

In [ ]:
data = load_s2l2a_with_offset_patch(
    dc, query | load_params
)
data

## Product definition

Display the measurement definitions for the selected product.

Use `list_measurements` to show the details for a product, and `masking.describe_variable_flags` to show the flag definitions.

See [Appendix](#Appendix) for a Table of Sentinel-2 ARD bands and corresponding product measurement and alias names.

In [ ]:
# Measurement definitions for the selected product
Measurement_info = dc.list_measurements().loc[query['product']]
Notebook_utils.heading(f'Measurement table for product: {query["product"]}')
display(measurement_info)

# Separate lists of measurement names and flag names
measurement_names = measurement_info[ pd.isnull(measurement_info.flags_definition)].index
flag_names        = measurement_info[pd.notnull(measurement_info.flags_definition)].index

notebook_utils.heading('Selected Measurement and Flag names')
display(pd.DataFrame({
    'group': ['Measurement names', 'Flag names'],
    'names': [', '.join(measurement_names), ', '.join(flag_names)]
}))

# Flag definitions
for flag in flag_names:
    notebook_utils.heading(f'Flag definition table for flag name: {flag}')
    display(masking.describe_variable_flags(data[flag]))

In [ ]:
flags_def = masking.describe_variable_flags(data[flag]).values
flags_def = flags_def.tolist()[0][1]
flags_def

## Quality layer

In [ ]:
# Make SCL flags image
flag_name = 'scl'
flag_data = data[[flag_name]].where(valid_mask[flag_name]).persist()   # Dataset
display(flag_data)

In [ ]:
# Make SCL image
# https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm
# https://www.sentinel-hub.com/faq/how-get-s2a-scene-classification-sentinel-2/

from bokeh.models.tickers import FixedTicker

color_def = [
    (0,  '#000000', 'No data'),   # black
    (1,  '#ff0004', 'Saturated or defective'),   # red
    (2,  '#868686', 'Dark features or shadows'),   # gray
    (3,  '#774c0b', 'Cloud shadows'),   # brown
    (4,  '#10d32d', 'Vegetation'),   # green
    (5,  '#ffff53', 'Not vegetated'),   # yellow
    (6,  '#0000ff', 'Water'),   # blue
    (7,  '#818181', 'Unclassified'),   # medium gray
    (8,  '#c0c0c0', 'Cloud medium probability'),   # light gray
    (9,  '#f2f2f2', 'cloud high probability'),   # very light gray
    (10, '#bbc5ec', 'Thin cirrus'),   # light blue/purple
    (11, '#53fff9', 'Snow or ice'),   # cyan
]
color_val = [x[0] for x in color_def]
color_hex = [x[1] for x in color_def]
color_txt = [f'{x[0]:2d}: {x[2]}' for x in color_def]
color_lim = (min(color_val), max(color_val) + 1)
bin_edges = color_val + [max(color_val) + 1]
bin_range = (color_val[0] + 0.5, color_val[-1] + 0.5)  # No idea why (0.5,11.5) works and (0,11) or (0,12) do not

# These options manipulate the color map and colorbar to show the categories for this product
options = {
    'title': f'Flag data for: {query["product"]} ({flag_name})',
    'cmap': color_hex,
    'clim': color_lim,
    'color_levels': bin_edges,
    'colorbar': True,
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'tools': ['hover'],
    'colorbar_opts': {
        'major_label_overrides': dict(zip(color_val, color_txt)),
        'major_label_text_align': 'left',
        'ticker': FixedTicker(ticks=color_val),
    },
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

quality_plot = flag_data.hvplot.image(
    x = 'x', y = 'y',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mode(),          # Datashader selects mode value, requires 'hv.Image'
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Datset crs
    projection = ccrs.PlateCarree(),         # Output projection (ccrs.PlateCarree() when coastline=True)
    coastline = '10m',                       # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = bin_range)

# display(quality_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(quality_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

In [ ]:
# Create Mask layer

good_pixel_flags = [flags_def[str(i)] for i in [4, 5, 6]]

good_pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)  # -> DataArray
display(good_pixel_mask)  # Type: bool

## Scaling and nodata

**TODO:** Update once the `load_s2l2a_with_offset_patch` is working, as this will have applied the scale and offset.

The cell below then only applies the mask, which can likely be combined with the cell above.

In [ ]:
# Select a layer and apply masking and scaling, then persist in dask

layer_name = 'nir'
layer_list = ['blue', 'red', 'nir', 'scl']

# Get scaling and offset values from product description
scale = measurement_info.loc[layer_name].scale_factor
offset = measurement_info.loc[layer_name].add_offset

# Apply valid mask and good pixel mask (single layer)
# layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask) * scale + offset
# layer = layer.persist()

# Apply valid mask and good pixel mask (multiple layers)
layer_set = data[layer_list].where(valid_mask[layer_list] & good_pixel_mask) * scale + offset
layer_set = layer_set.persist()
layer_set

## Visualisation

In [ ]:
# Generate a plot

options = {
    'title': f'{query["product"]}: {layer_name}',
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'cmap': cc.rainbow,
    'clim': (0, 1),                          # Limit the color range depending on the layer_name
    'colorbar': True,
    'tools': ['hover'],
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

layer_plot = layer.hvplot.image(
    x = 'x', y = 'y',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                        # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

## Optional: Reproject

Use xr_reproject to reproject to UTM data to a new project, e.g. to align with other datasets

See [dea-notebooks/Frequently_used_code/Reprojecting_data.ipynb](#../../../dea-notebooks/Frequently_used_code/Reprojecting_data.ipynb) for additional examples.

In [ ]:
# This example uses the query bounding box and epsg:4326  as the target

target_crs = 'epsg:4326'
target_res = (0.0002, 0.0002)   # approx. 20 m

target = GeoBox.from_geopolygon(box(min_longitude, min_latitude, max_longitude, max_latitude, target_crs), target_res)
layer_geo = xr_reproject(layer, target)

In [ ]:
# Set the Dataset CRS
plot_crs = target_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used
    
layer_plot = layer_geo.hvplot.image(
    x = 'longitude', y = 'latitude',         # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

## Appendix

#### EC Copernicus Data Hub

https://scihub.copernicus.eu

EC Copernicus Data Hub is the primary source of S2 L1B and L2A (Sen2cor) data. L2A (Sen2cor) is available globally from December 2018 onwards.

Atmospheric correction - Sen2cor
- Detailed summary of bands and thresholds for classification: https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm
- Details of Sen2cor paramaters: https://sentinels.copernicus.eu/documents/247904/685211/Sen2-Cor-L2A-Input-Output-Data-Definition-Document

Atmospheric correction - Any other method, including Sen2cor prior to December 2018
- EASI can inject just about any preprocessing routine into our data preparation pipeline. That is, we would download S2 L1B and apply a preprocessing routine (an atmospheric correction method). This would create a new product name like ```cophub_s2_[type]_[routine]```.

#### Sentinel-2 Cloud-Optimized GeoTIFFs

https://registry.opendata.aws/sentinel-2-l2a-cogs/

Open and free S2 L2a (Sen2cor) COGs. Same time range as from CopHub (December 2018 onwards).

TODO: Check the latency between CopHub and AWS.